In [6]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import locale
locale.setlocale(locale.LC_ALL, 'ID')
import re

In [36]:
def getIndeksLink(links, page, cat_link, category, date=datetime.strftime(datetime.today(), '%m/%d/%Y')):
    """
    Untuk mengambil seluruh url 
    link pada indeks category tertentu
    date format : dd/mm/YYYY
    """
    print("page ", page)
    if cat_link == 'news':
        url = "https://"+cat_link+".detik.com/indeks/all/"+str(page)+"?date="+date
    else :
        url = "https://"+cat_link+".detik.com/indeks/"+str(page)+"?date="+date
    print(url)
    # Make the request and create the response object: response
    response = requests.get(url)
    # Extract HTML texts contained in Response object: html
    html = response.text
    # Create a BeautifulSoup object from the HTML: soup
    soup = BeautifulSoup(html, "html5lib")
    contentDiv = soup.find('ul', attrs={'id':'indeks-container'})
    indeks = contentDiv.findAll('article')
    for post in indeks:
        link = [post.find('a', href=True)['href'], category]
        links.append(link)
        
    el_page = soup.find('div', class_="paging paging2")
    if el_page:
        max_page = int(soup.find('div', class_="paging paging2").findAll('a')[-2].text.replace('\n', '').strip(' '))
    
        if page < max_page:
            links = getIndeksLink(links, page+1, cat_link, category, date)
        
    return links

In [122]:
def getDetailBerita(links):
    all_articles = []
    for link in links:
        articles = {}
        #link
        url = link[0]
        response = requests.get(url)
        html = response.text
        # Create a BeautifulSoup object from the HTML: soup
        soup = BeautifulSoup(html, "html5lib")
        
        #extract subcategory from breadcrumb
        bc = soup.find('div', class_="breadcrumb")
        if not bc:
            continue
        
        sub = bc.findAll('a')[1].text
        if ("foto" in sub.lower()) or  "video" in sub.lower(): 
            continue
            
        articles['subcategory'] = sub
        
        articles['id'] = int(soup.find("meta", attrs={'name':'articleid'})['content'])
        #category
        articles['category'] = link[1]
        articles['url'] = url
        
        article = soup.find('article')
        
        #extract date
        pubdate = soup.find("meta", attrs={'name':'publishdate'})['content']
        pubdate = pubdate.strip(' \t\n\r')
        articles['pubdate'] = datetime.strftime(datetime.strptime(pubdate, "%Y/%m/%d %H:%M:%S"), '%Y-%m-%d %H:%M:%S')
        
        #extract author
        articles['author'] = soup.find("meta", attrs={'name':'author'})['content']
        
        #extract title
        articles['title'] = article.find('div', class_="jdl").find('h1').text
        
        #source
        articles['source'] = 'detik'
        
        #extract comments count
        articles['comments'] = int(soup.find('a', class_="komentar").find('span').text.replace('Komentar', '').strip(' \t\n\r'))
        
        #extract tags
        tags = article.find('div', class_="detail_tag").findAll('a')
        articles['tags'] = ','.join([x.text for x in tags])
        
        #extract images
        articles['images'] = article.find('div', class_="pic_artikel").find('img')['src']
        
        #extract detail
        detail = article.find('div', class_="detail_text")
        
        #hapus link sisip
        for link in detail.findAll('table', class_="linksisip"):
            link.decompose()
        
        #hapus video sisip
        for tag in detail.findAll('div', class_="sisip_embed_sosmed"):
            tag.decompose()
            
        #hapus all setelah clear fix
        for det in detail.find('div', class_="clearfix mb20").findAllNext():
            det.decompose()
            
        #hapus all script
        for script in detail.findAll('script'):
            script.decompose()
        
        #extract content
        detail = BeautifulSoup(detail.decode_contents().replace('<br/>', ' '), "html5lib")
        content = re.sub(r'\n|\t|\b|\r','',detail.text)
        articles['content'] = re.sub(r'(Tonton juga).*','', content)
        print('memasukkan berita id ', articles['id'])
        all_articles.append(articles)
    return all_articles

In [42]:
links = []

In [53]:
links = getIndeksLink(links, 1, 'news', 'news')

page  1
https://news.detik.com/indeks/all/1?date=07/27/2018
page  2
https://news.detik.com/indeks/all/2?date=07/27/2018
page  3
https://news.detik.com/indeks/all/3?date=07/27/2018
page  4
https://news.detik.com/indeks/all/4?date=07/27/2018
page  5
https://news.detik.com/indeks/all/5?date=07/27/2018
page  6
https://news.detik.com/indeks/all/6?date=07/27/2018
page  7
https://news.detik.com/indeks/all/7?date=07/27/2018
page  8
https://news.detik.com/indeks/all/8?date=07/27/2018


In [123]:
detail = getDetailBerita(links[10:13])

memasukkan berita id  4136427
memasukkan berita id  4136402


In [124]:
detail

[{'subcategory': 'Berita',
  'id': 4136427,
  'category': 'news',
  'url': 'https://news.detik.com/berita/d-4136427/priit-pelanggar-ganjil-genap-ditilang-mulai-1-agustus',
  'pubdate': '2018-07-27 13:00:00',
  'author': 'Peti',
  'title': 'Priit! Pelanggar Ganjil-Genap Ditilang Mulai 1 Agustus',
  'source': 'detik',
  'comments': 0,
  'tags': 'asian games 2018,ganjil genap,ganjil genap asian games,ganjil genap diperluas',
  'images': 'https://akcdn.detik.net.id/community/media/visual/2018/07/18/d7868207-2c18-4a20-a933-9aa4692c4dcb_169.jpeg?w=780&q=90',
  'content': 'Jakarta - Uji coba dan sosialisasi perluasan ganjil-genap akan berakhir pada 31 Juli 2018. Polisi akan mulai menindak pelanggar aturan ganjil-genap pada 1 Agustus 2018.  "Sudah fix, jadi mulai 1 Agustus akan dilaksanakan penindakan. Jadi kalau sekarang kan masih sosialisasi, kemudian yang melanggar disuruh keluar dari jalur itu," ujar Direktur Lalu Lintas Polda Metro Jaya Kombes Yusuf kepada wartawan di Mapolda Metro Jaya, 

In [112]:
import pandas as pd

In [113]:
df = pd.DataFrame(detail)

In [115]:
df.to_csv('tes.csv')

In [116]:
df

,author,category,comments,content,id,images,pubdate,source,subcategory,tags,title,url
0,Peti - detikNews,news,0,Jakarta - Uji coba dan sosialisasi perluasan g...,4136427,https://akcdn.detik.net.id/community/media/vis...,2018-07-27 13:00:00,detik,Berita,"asian games 2018,ganjil genap,ganjil genap asi...",Priit! Pelanggar Ganjil-Genap Ditilang Mulai 1...,https://news.detik.com/berita/d-4136427/priit-...
1,Muhajir Arifin - detikNews,news,0,Pasuruan -Polisi dan Dinas Perhubungan sepakat...,4136402,https://akcdn.detik.net.id/community/media/vis...,2018-07-27 12:55:53,detik,Berita-jawa-timur,"traffic light,purutrejo,pasuruan",Durasi Nyala Traffic Light Perempatan Purutrej...,https://news.detik.com/berita-jawa-timur/d-413...


In [138]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [ ]:
options = Options()
options.add_argument('--headless')
options.add_argument('--disable-gpu')  # Last I checked this was necessary.
# options.add_argument('--disable-extensions')

driver = webdriver.Chrome("../chromedriver.exe", chrome_options=options)

In [144]:
driver.get("https://nasional.kompas.com/read/2018/07/27/12344561/adiknya-ditangkap-kpk-zulkifli-mohon-maaf")
html = driver.page_source

In [147]:
soup = BeautifulSoup(html, 'html5lib')

In [148]:
soup.find('div', class_="span4 comments-count tright")